In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [20]:
class Player:
    def __init__(self, position, score=0):
        self.position = position
        self.score = score
    def update_position(self, moves):
        pos = (self.position + sum(moves))%10 
        pos = 10 if pos ==0 else pos
        self.position = pos
        
    def update_score(self):
        self.score += self.position
        
        

In [37]:

def play_dirac_dice(init_positions):
    
    def dice_roll(total_roll):
        rolls=[]
        for _ in range(3):
            total_roll+=1
            val = total_roll%100
            val = 100 if val == 0 else val
            rolls.append(val)
        return rolls, total_roll
    
    players= [ Player(i,0) for i in init_positions]
    tot_roll = 0
    while players[0].score < 1000 and players[1].score < 1000:
        rolls,tot_roll=dice_roll(tot_roll)
        players[0].update_position(rolls)
        players[0].update_score()
        if players[0].score >=1000:
            break
        rolls,tot_roll=dice_roll(tot_roll)
        players[1].update_position(rolls)
        players[1].update_score()
        
    return min(players[0].score,players[1].score)*tot_roll

In [38]:
play_dirac_dice([4,8])

739785

In [39]:
play_dirac_dice([10,3])

742257

# Part 2 
## Quantum splitting

In [41]:
sum_counts = [0 for _ in range(10)]
for i in range(1,4):
    for j in range(1,4):
        for k in range(1,4):
            sum_counts[i+j+k]+=1

In [43]:
len( {'1':2, '2':3})

2

In [230]:
class QuantumDice:
    
    def __init__(self, init_positions):
        players = [ (p,0) for p in init_positions]
       
        key = self._create_key(players)
        self.games_in_flight={
           (key) : 1 
        }
        
        self.player_one_games = 0
        self.player_two_games = 0
        self.dice_sum_cnts = [ 
                          (3,1), 
                          (4,3), 
                          (5,6), 
                          (6,7),  
                          (7,6), 
                          (8,3), 
                          (9,1) ]
        self.MAX = 21
        
    def _create_key(self, players):
        return '|'.join(['_'.join([str(i) for i in p]) for p in players])
    
    def _read_key(self, key):
        players = key.split('|')
        return [ [int(i) for i in p.split('_')] for p in players]
        
    
    def _calc_new_pos(self,pos, roll):
        new_pos = (pos+roll)%10
        return 10 if new_pos == 0 else new_pos
        
    def run_games(self):
        in_flight = self.games_in_flight.copy()
        
        while len(in_flight) >0:
            next_round={}
            for players, game_cnt in in_flight.items():
                players = self._read_key(players)
                
                for roll, dice_count in self.dice_sum_cnts:
                    p1_n_universes = game_cnt*dice_count
                    player_one = players[0]
                    pos_one = self._calc_new_pos(player_one[0], roll)
                    new_score_one = player_one[1] + pos_one
                    
                    if new_score_one >=self.MAX :
                        self.player_one_games += p1_n_universes
                        continue
                        
                    new_player_one= (pos_one,new_score_one)
                    
                    for roll_2, dice_count_2 in self.dice_sum_cnts:
                        p2_n_universes = p1_n_universes*dice_count_2
                        
                        player_two = players[1]
                        pos_two = self._calc_new_pos(player_two[0], roll_2)
                        new_score_two = player_two[1] + pos_two
                        
                        if new_score_two >=self.MAX :
                            self.player_two_games += p2_n_universes 
                            continue
                        new_player_two= (pos_two,new_score_two)
                        
                        new_key = self._create_key([new_player_one,new_player_two])
                        try:
                            next_round[new_key] = next_round[new_key]+ p2_n_universes 
                        except:
                            next_round[new_key] = p2_n_universes
            
            in_flight = next_round.copy()
                        
        self.games_in_flight = in_flight
                        
    def get_winner(self):
        self.player_one_games = 0
        self.player_two_games = 0
        self.run_games()
        
        return max(self.player_one_games, self.player_two_games)
        
                        
                    
                    

In [231]:
test = QuantumDice([4,8])

In [232]:
test.run_games()

In [233]:
test.player_one_games

444356092776315

In [234]:
test.player_two_games

341960390180808

In [235]:
prod = QuantumDice([10,3])

In [236]:
prod.get_winner()

93726416205179